In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [2]:
import DualwormFunctions as dw
import Observables as obs

%load_ext line_profiler
%load_ext memory_profiler

import RunBasis
import argparse
import StartStates as strst
import numpy as np

import AnalysisFunctions as af # functions to make the analysis code cleaner
import KagomeDrawing as kdraw


import matplotlib.pyplot as plt
%matplotlib inline

import yep

In [3]:
nst = 512;
nsm = 1024;
nips = 4;
nrps = 4;
nb = 32;
measperiod = 1;
nt = 128
nh = 1
ncores = 4
nmaxiter = 1
L = 4
J1 = 1
J2 = 0.2
J3 = 0.15
h = 0
stroutput = "JustSomeJ1{0}J2{1}J3{2}_L{3}_family_step1".format(J1,J2,J3,L)

In [4]:
def genArgs():
    parser = argparse.ArgumentParser()
    
    parser.add_argument('-f', type = str)

    parser.add_argument('--L', type = int, default = L, help = 'Lattice side size')

    # COUPLINGS
    parser.add_argument('--J1', type = float, default = J1,
                        help = 'NN coupling') # nearest-neighbour coupling
    parser.add_argument('--J2', type = float, default = J2,
                        help = '2nd NN coupling') # 2nd NN coupling
    parser.add_argument('--J3', type = float, default = J3,
                        help = '3rd NN coupling') # 3rd NN coupling
    parser.add_argument('--J4', type = float, default = 0.0,
                        help = '4th NN coupling') 
    #NUMBER OF STEPS AND ITERATIONS
    parser.add_argument('--nst', type = int, default = nst,
                        help = 'number of thermalisation steps') # number of thermalisation steps
    parser.add_argument('--nsm', type = int, default = nsm,
                        help = 'number of measurements steps') # number of measurement steps
    parser.add_argument('--nips', type = int, default = nips,
                        help = 'number of worm constructions per MC step')
    parser.add_argument('--nrps', type = int, default = nrps,
                        help = 'number of replica loops per MC step')
    parser.add_argument('--measperiod', type = int, default = measperiod,
                        help = 'number of nips worm building + swaps between measurements')
    parser.add_argument('--nb', type = int, default = nb,
                        help = 'number of bins')

    #PARALLELISATION
    parser.add_argument('--ncores', type = int, default = ncores,
                        help = 'number of threads to use')

    
    # INITIALISATION PARAMETERS
    parser.add_argument('--randominit', default = True, action ='store_true',
                        help = 'intialise the states randomly')
    parser.add_argument('--same', default = False, action = 'store_true',
                        help = '''initialise all temperatures with the same
                        state (debug purposes)''')
    parser.add_argument('--magninit', default = False, action = 'store_true',
                        help = '''initialise all the temperature with
                        one of the m=1/3 GS''')
    parser.add_argument('--magnstripes', default = False, action = 'store_true',
                       help = '''initialise all the temperature with
                       m=1/3 stripes''')
    parser.add_argument('--maxflip', default = False, action = 'store_true',
                       help = '''initialise all the temperature with
                       maximally flippable plateau''')
    parser.add_argument('--loadfromfile', default = False, action = 'store_true',
                       help = '''initialise all the states with
                       results from a previously performed simulations''')
    parser.add_argument('--filename', type = str, default = '',
                       help = '''initialise all the states with
                       results from a previously performed simulations''')
    
    #WORM PARAMETERS
    parser.add_argument('--nmaxiter', type = int, default = nmaxiter,
                        help = '''maximal number of segments in a loop update over the
                        size of the lattice (1 = 1times the number of dualbonds in the
                        lattice)''')
    parser.add_argument('--measupdate', default = False, action = 'store_true',
                       help = '''activate to mimic the action of the measuring tip''')
    parser.add_argument('--p', type = float, default = 0.0, 
                       help = '''prob of the measuring tip flipping the spin (number between 0 and 1)''')
    parser.add_argument('--ssf', default = False, action = 'store_true',
                        help = 'activate for single spin flip update')
    parser.add_argument('--alternate', default = True, action = 'store_true',
                        help = 'activate for single spin flip update and dw update')
    parser.add_argument('--checkgs', default = True, action = 'store_true',
                        help = 'activate to debug ssf')
    
    #TEMPERATURE PARAMETERS
    parser.add_argument('--t_list', nargs = '+', type = float, default = [0.001,60],
                        help = 'list of limiting temperature values')
    parser.add_argument('--nt_list', nargs = '+', type = int, default = [nt],
                        help = 'list of number of temperatures in between the given limiting temperatures')
    parser.add_argument('--log_tlist', default = True, action='store_true',
                        help = 'state whether you want the temperature be spaced log-like or linear-like (activate if you want log)')
    parser.add_argument('--stat_temps_lims', nargs = '+', type = float,
                        help = '''limiting temperatures for the various ranges of
                        measurements''') 
                        #default will be set to none, and then we can decide what to do later on.
    #MAGNETIC FIELD PARAMETERS
    parser.add_argument('--h_list', nargs = '+', default = [0,h],type = float,
                        help = 'list of limiting magnetic field values')
    parser.add_argument('--nh_list', nargs = '+', default = [nh], type = int,
                        help = 'list of number of magnetic fields in between the given limiting temperatures')
    parser.add_argument('--stat_hfields_lims', nargs = '+', type = float,
                help = '''limiting magnetic fields for the various ranges of
                measurements''') 
    #MEASUREMENTS PARAMETERS
    parser.add_argument('--generatingMode', default = True, action = 'store_true',
                        help = 'activate if you want to generate a number of ground states and low energy states')
    parser.add_argument('--energy', default = True, action = 'store_true',
                        help = 'activate if you want to save the energy')
    parser.add_argument('--magnetisation', default = False, action = 'store_true',
                        help = 'activate if you want to save the magnetisation')
    parser.add_argument('--charges', default = False, action = 'store_true',
                        help = 'activate if you want to save the charges')
    parser.add_argument('--correlations', default = False, action = 'store_true',
                        help = 'activate if you want to save either central or all correlations')
    parser.add_argument('--all_correlations', default = False, action = 'store_true',
                        help = '''activate if you want to save the correlations for all non-equivalent
                        pairs of sites. Otherwise, will save central correlations.''')
    parser.add_argument('--firstcorrelations', default = False, action = 'store_true',
                        help = 'activate if you want to save first correlations, otherwise will save central')
    #SAVE
    parser.add_argument('--output', type = str, default = stroutput, help = 'saving filename (.pkl will be added)')
    args = parser.parse_args()
    return args

In [ ]:
def callRunBasis():
    args = genArgs()
    return RunBasis.main(args)

In [ ]:
#%%prun -s  -T prun0
#yep.start(stroutput+'.log')
%lprun -f dw.mcs_swaps [meanstat, swapst, swapsh, failedupdatesth, failedupdates] = callRunBasis()
#yep.stop()

-------------------Initialisation--------------------
./JustSomeJ11J20.2J30.15_L4_family_step1_folder/backup.hkl
Lattice side size:  4
J1  1
J2  0.2
J3  0.15
J3st  0.15
Couplings extracted
Hamiltonian expression (without field) computed
alternating ssf and dw update
Number of temperatures:  128
Temperatures: [1.00000000e-03 1.09049389e-03 1.18917693e-03 1.29679018e-03
 1.41414178e-03 1.54211297e-03 1.68166478e-03 1.83384518e-03
 1.99979697e-03 2.18076638e-03 2.37811242e-03 2.59331708e-03
 2.82799644e-03 3.08391285e-03 3.36298813e-03 3.66731802e-03
 3.99918791e-03 4.36109000e-03 4.75574202e-03 5.18610763e-03
 5.65541870e-03 6.16719957e-03 6.72529347e-03 7.33389146e-03
 7.99756386e-03 8.72129456e-03 9.51051846e-03 1.03711623e-02
 1.13096892e-02 1.23331470e-02 1.34492215e-02 1.46662939e-02
 1.59935040e-02 1.74408184e-02 1.90191060e-02 2.07402189e-02
 2.26170821e-02 2.46637900e-02 2.68957123e-02 2.93296101e-02
 3.19837607e-02 3.48780958e-02 3.80343505e-02 4.14762270e-02
 4.52295722e-02 4.9

In [ ]:
# Create a folder to save the pictures
foldername = "./" + stroutput + "_folder/" 
results_foldername = 'Results/'
os.makedirs(foldername + 'Plots' + results_foldername, exist_ok = True)

filenamelist = ["backup"]
n = len(filenamelist)

In [ ]:
[L, numsites, J1, J2, J3, J3st, J4, nb, num_in_bin, temperatures, nt,
 stat_temps, temperatures_plots, hfields, nh,
 stat_hfields, hfields_plots, listfunctions, sref] =\
af.LoadParameters(foldername, filenamelist)

In [ ]:
kw = {'binning': False, 'plzplot': True, 'plotmin': 0, 'plotmax': 16}
[t_h_MeanE, t_h_MeanEsq, t_h_varMeanE, t_h_varMeanEsq, C, ErrC] = \
 af.LoadEnergy(foldername, filenamelist, numsites,
               nb, stat_temps, temperatures, stat_hfields,
               listfunctions, **kw)


In [ ]:
print(t_h_MeanE[0][0])
print((-2/15*J1[0] - 2/3*J2[0] - J3[0]))

In [ ]:
tidmin = 0
tidmax = [len(stat_temps[i]) for i in range(n)]
temperatures_plots = np.array(temperatures_plots)
### Energy
S0 = np.log(2)
kw = {'gscheck': True}
af.BasicPlotsE(L, n, tidmin, tidmax, temperatures_plots, hfields_plots, foldername,
                results_foldername, filenamelist, t_h_MeanE, t_h_MeanEsq, t_h_varMeanE,
                t_h_varMeanEsq, C, ErrC, J1, J2, J3, J4, **kw)
plt.show()

In [ ]:
color_frustrated = "red"
color_satisfied = "lightgreen"
i = 0

In [ ]:
import hickle as hkl

In [ ]:
import MFUPreparation as mfu

In [ ]:
import importlib

In [ ]:
importlib.reload(mfu)

In [ ]:
kw = {'binning': False, 'plzplot': True, 'plotmin': 0, 'plotmax': 16}
[groundspinstates, groundstates, groundcharges] = \
 af.LoadGroundStates(foldername, filenamelist, L, nh, [nsm*nips//measperiod],**kw)
[d_ijl, ijl_d, s_ijl, ijl_s, d_2s, s2_d, d_nd, d_vd, d_wn,
 sidlist, didlist, c_ijl, ijl_c, c2s, csign] =\
dw.latticeinit(L[0])
(s_ijl, ijl_s) = kdraw.createspinsitetable(L[0])


backup = "./"+foldername+filenamelist[0]+".hkl"
couplings = hkl.load(backup, path="/parameters/couplings")
hamiltonian = dw.Hamiltonian(couplings, d_ijl, ijl_d, L[0])

gsenergy = -2/3 * J1[i] - 1/3 * J3[i]
latsize = len(s_ijl)
fam, spinfam = mfu.FamiliesFromStates(hamiltonian, groundstates[0],
                                      gsenergy, groundspinstates[0],
                                      d_wn, latsize, 0.5, d_ijl)
#for it in range(nsm):
#    energy = dw.compute_energy(hamiltonian, groundstates[0][it], latsize = len(s_ijl))
#    print("Phase 1: ",af.testPhase(energy,  (-2/3 * J1[i] - 2/3 * J2[i] + J3[i])))
#    print("Phase 2: ", af.testPhase(energy,  (-2/3 * J1[i] - 2/3 * J2[i] + 3 * J3[i])))
#    print("Phase 3: ", af.testPhase(energy,  (-2/3 * J1[i] - 1/3 * J3[i])))
#    print("Phase 4: ", af.testPhase(energy, (-2/3 * J1[i] + 2/3 * J2[i] - J3[i])))
#    print("Phase 5: ", af.testPhase(energy,   (-2/3 * J1[i] + 2 * J2[i] - J3[i])))
#    print("Phase 6: ", af.testPhase(energy,   (-2/9 * J1[i] - 2/3 * J2[i] - 7/9 * J3[i])))
#    print("Phase 7: ", af.testPhase(energy,  (-2/15 * J1[i] - 2/3 * J2[i] - J3[i])))
#    print("Phase 8: ", af.testPhase(energy,   (2/3 * J1[i] - 2/3 * J2[i] - J3[i])))
#    print("Phase 9: ", af.testPhase(energy,   (2/3 * J1[i] - 2/3 * J2[i] + 1/3 * J3[i])))
#    print("Phase 10: ", af.testPhase(energy,   (6/7 * J1[i] - 2/7 * J2[i] - J3[i])))
#    print("Phase 11: ", af.testPhase(energy,   (2 * J1[i] + 2 * J2[i] + 3 * J3[i])))
#
#    kdraw.plotstate(it, L[0], d_ijl, ijl_s, sidlist, didlist, s2_d,
#               groundstates[0], groundspinstates[0],dim_node_color = 'orange',
#                dim_color = "orange", no_dim_color = 'lightgrey')
#    kdraw.plot_kag_spinstate(np.array(groundspinstates[0][it]), ijl_s, L[0], 2, 'lightblue', 'lightblue', 'peachpuff', linewidth = 1)
#    kdraw.plot_honeycomb_chargestate(groundcharges[0][it], ijl_c, L[0], 2, c1_color = color_frustrated, 
#                                     c2_color = color_satisfied, c3_color = color_satisfied, c4_color = color_frustrated)
#    plt.tick_params(  
#        which = 'both',      # both major and minor ticks are affected
#        bottom = False,      # ticks along the bottom edge are off
#        top = False,         # ticks along the top edge are off
#        labelbottom = False,
#        left = False,
#        right = False,
#        labelleft = False)
#    plt.legend()
#    plt.show()

In [ ]:
fam

In [ ]:
spinfam

In [ ]:
for family in fam:
    for index in family:
        kdraw.plotstate(index, L[0], d_ijl, ijl_s, sidlist, didlist, s2_d,
                   groundstates[0], groundspinstates[0],dim_node_color = 'orange',
                    dim_color = "orange", no_dim_color = 'lightgrey')
        kdraw.plot_kag_spinstate(np.array(groundspinstates[0][index]), ijl_s, L[0], 2, 'lightblue', 'lightblue', 'peachpuff', linewidth = 1)
        kdraw.plot_honeycomb_chargestate(groundcharges[0][index], ijl_c, L[0], 2, c1_color = color_frustrated, 
                                         c2_color = color_satisfied, c3_color = color_satisfied, c4_color = color_frustrated)
        plt.tick_params(  
            which = 'both',      # both major and minor ticks are affected
            bottom = False,      # ticks along the bottom edge are off
            top = False,         # ticks along the top edge are off
            labelbottom = False,
            left = False,
            right = False,
            labelleft = False)
        plt.legend()
        plt.show()